['id', 'pageurl', 'eid', 'mid', 'event_name', 'event_org', 'event_date',
       'event_place', 'f1pageurl', 'f2pageurl', 'f1name', 'f2name', 'f1result',
       'f2result', 'f1fid', 'f2fid', 'method', 'method_d', 'ref', 'round',
       'duration', 'id', 'url', 'fid', 'name', 'nick', 'birth_date', 'height',
       'weight', 'association', 'class', 'locality', 'country', 'id', 'url',
       'fid', 'name', 'nick', 'birth_date', 'height', 'weight', 'association',
       'class', 'locality', 'country']

In [1]:
import pandas as pd
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import psycopg2


try:
    conn = psycopg2.connect("dbname='samkreter' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()
df = pd.read_sql_query("select  fights.f1result, f1.birth_date AS f1_birth_date, f1.height AS f1_height, \
       f1.weight AS f1_weight, f2.birth_date AS f2_birth_date, f2.height AS f2_height, f2.weight AS f2_weight \
        from fights INNER JOIN fighters AS f1 ON fights.f1name = f1.name INNER JOIN fighters AS f2 ON fights.f2name = f2.name;", conn)
conn.close()

In [2]:
df.head()

,f1result,f1_birth_date,f1_height,f1_weight,f2_birth_date,f2_height,f2_weight
0,win,None,0,155,None,0,0
1,win,1974-12-06,67,135,1969-07-16,72,160
2,win,1974-07-26,72,169,1973-05-20,70,170
3,win,1974-07-26,72,169,1973-05-20,70,170
4,win,1974-12-06,67,135,1978-12-13,69,145


In [3]:
import datetime as DT
from dateutil.relativedelta import relativedelta


pd.set_option('display.max_columns', None)

In [4]:
#Convert results to numbers 
df['f1result'] = df['f1result'].replace(['win', 'draw', 'NC'],[1,2,2])


In [8]:
#Convert the birthdate to an age############
now = DT.datetime.now().date()
def t(x):
    try:
        return int((now - x).days / 365)
    except:
        return 0
    
df["f1_age"] = df["f1_birth_date"].apply(t)
df["f2_age"] = df['f2_birth_date'].apply(t)


del df['f1_birth_date']
del df['f2_birth_date']

df.head()

,f1result,f1_height,f1_weight,f2_height,f2_weight,f1_age,f2_age
0,1,0,155,0,0,0,0
1,1,67,135,72,160,42,47
2,1,72,169,70,170,42,43
3,1,72,169,70,170,42,43
4,1,67,135,69,145,42,38


In [10]:
df2 = df.copy()
tmp = df2[["f1_height","f1_weight","f1_age"]]
df2[["f1_height","f1_weight","f1_age"]] = df2[["f2_height","f2_weight","f2_age"]]
df2[["f2_height","f2_weight","f2_age"]] = tmp

df2['f1result'] = 0


#Create the labels
y1 = df['f1result']
y2 = df2['f1result']

del df['f1result']
del df2['f1result']

y = y1.append(y2)
X = df.append(df2)

X.head()
#y.head()

,f1_height,f1_weight,f2_height,f2_weight,f1_age,f2_age
0,0,155,0,0,0,0
1,67,135,72,160,42,47
2,72,169,70,170,42,43
3,72,169,70,170,42,43
4,67,135,69,145,42,38


In [11]:
from sklearn import datasets
# Our Classifier
from sklearn.naive_bayes import GaussianNB
# Get the accuracy score of the final model
from sklearn.metrics import accuracy_score
# Split the training set into train and test
from sklearn.cross_validation import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.5,random_state = 1)

nbc = GaussianNB()

nbc.fit(X_train,y_train)

preds = nbc.predict(X_test)



print("Accuracy of the model is: %.2f%%" % (accuracy_score(preds,y_test) * 100))



Accuracy of the model is: 51.65%


In [ ]:
stat1 = pd.read_csv("sherdog-fighters.csv")
stat1.head()

stat1['Weight (KG)'].unique()